# Remediator

> The main Remediator agent that orchestrates the resolution of IRIS instance problems using IrisDb and Customer agents.

This module implements the Remediator using the OpenAI SDK Agents framework.

In [8]:
#| default_exp remediator

In [9]:
#| hide
from nbdev.showdoc import *

In [10]:
#| export
from typing import Dict, List, Optional, Any
from dataclasses import dataclass, field
from enum import Enum
import json
import os
from openai import OpenAI, AsyncOpenAI

## Data Structures

Define the structured formats for communication between agents.

In [11]:
#| export
class ProblemType(Enum):
    """Types of problems that can occur in IRIS instance."""
    LICENSE = "license"
    CONFIGURATION = "configuration"
    OS_RESOURCE = "os_resource"
    UNKNOWN = "unknown"

In [12]:
#| export
@dataclass
class Problem:
    """Structured representation of a problem detected in IRIS instance."""
    problem_type: str
    description: str
    severity: str  # 'critical', 'high', 'medium', 'low'
    log_entry: Optional[str] = None
    affected_component: Optional[str] = None
    
    def to_dict(self) -> Dict[str, Any]:
        return {
            'problem_type': self.problem_type,
            'description': self.description,
            'severity': self.severity,
            'log_entry': self.log_entry,
            'affected_component': self.affected_component
        }
    
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'Problem':
        return cls(**data)

In [13]:
#| export
@dataclass
class Resolution:
    """Possible resolution for a problem."""
    action: str
    description: str
    requires_restart: bool = False
    parameters: Dict[str, Any] = field(default_factory=dict)
    
    def to_dict(self) -> Dict[str, Any]:
        return {
            'action': self.action,
            'description': self.description,
            'requires_restart': self.requires_restart,
            'parameters': self.parameters
        }
    
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'Resolution':
        return cls(**data)

## Remediator Agent

The main Remediator agent implemented using the OpenAI SDK Agents framework.

In [ ]:
#| export
from agents import Agent, OpenAIChatCompletionsModel

class RemediatorAgent:
    """Main Remediator agent using OpenAI SDK Agents framework."""
    
    def __init__(self, model):
        """Initialize the Remediator Agent.
        
        Args:
            model: Model to use (string for default models or OpenAIChatCompletionsModel instance for custom endpoints)
        """
        self.history: List[Dict[str, Any]] = []
        
        self.agent = Agent(
            name="Remediator",
            model=model,
            instructions="Resolve problems with InterSystems IRIS instance."
        )
    
    def log_step(self, step: str, data: Any) -> None:
        """Log a workflow step."""
        self.history.append({'step': step, 'data': data})
    
    async def remediate(self, prompt: str) -> str:
        """Main remediation workflow."""
        self.log_step('remediate_start', {'prompt': prompt})
                
        try:
            response = await self.agent.run(prompt)
            report = str(response)
            self.log_step('remediate_complete', {'report': report})
            return report
        except Exception as e:
            error_msg = f"Error: {str(e)}"
            self.log_step('remediate_error', {'error': error_msg})
            return error_msg

In [ ]:
#| export
def get_default_config():
    """Get default configuration for the Remediator Agent.
    
    Returns:
        OpenAIChatCompletionsModel: Configured model object ready to use with custom endpoint
    """
    api_key = os.getenv("OPENAI_API_KEY")
    # model_name = "gpt-5-mini"  # Use available model on internal API
    model_name = "openai/gpt-oss-120b"  # Use available model on internal API
    base_url = "https://plaza.iscinternal.com/genai/v1"
    
    client = AsyncOpenAI(base_url=base_url, api_key=api_key)
    model_obj = OpenAIChatCompletionsModel(model=model_name, openai_client=client)
    
    return model_obj

## Usage Example

In [16]:
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
# Example: Create a Problem instance
problem = Problem(
    problem_type="license",
    description="License has expired",
    severity="critical"
)
print(f"Problem: {problem.description}")

Problem: License has expired


In [ ]:
# Example: Create and use RemediatorAgent
# Note: Requires OPENAI_API_KEY environment variable in .env file

import asyncio

# Get configured model
model = get_default_config()

# Create the agent
remediator = RemediatorAgent(model=model)

# Run remediation (async example)
async def test_remediation():
    result = await remediator.remediate("Resolve errors with the IRIS instance")
    print("Remediation result:", result)
    return result

# Uncomment to run:
# asyncio.run(test_remediation())

print("RemediatorAgent configured with custom endpoint model")

RemediatorAgent configured with:
  Model: openai/gpt-oss-120b
  Base URL: https://plaza.iscinternal.com/genai/v1
  API Key: ***


## Export

In [19]:
#| hide
import nbdev; nbdev.nbdev_export()